## Checkpoint variants

A checkpoint variant is usually **a checkpoint where it's weights** are:

* Stored in a different floating point type for `lower precision` and `lower storage`, such as `torch.float16`, because it only requires half the bandwidth and storage to download. You cann't use this variant if you're continuing training or using a CPU.

* `Non-exponential mean averaged(EMA)` weights which shouldn't be used for inference. You should use these to continue `finetuning a model`.

Otherwise, a variant is `identical` to the original checkpoint. They have exactly the same seialization format, model structure, and weights have identical tensor shapes.

checkpoint type | weight name | argument for loading weights
--- | --- | ---
original | diffusion_pytorch_model.bin |
floating point | diffusion_pytorch_fp16_model.bin | variant, torch_dtype
non-EMA| diffusion_pytorch_ema_model.non_ema.bin | variant


There are two important arguments to know for loading variants:
* `torch_dtype` defines the floating point precision od the loaded checkpoints. For example, if you want to save bandwidth by loading a fp16 variant, you should specify torch_dtype=torch.float16 to `convert the weights` to fp16. Otherwise, the fp16 weights are convertd to the default fp32 precision. You can also load the original checkpoint without defining the variant argument, and convert it to fp16 with torch_dtype=torch.float16. In this case, the default fp32 weights are downloaded first, and then they'are converted to fp16 after loading.

* `variant` defines which files should be loaded from the repository. For example, if you want to load a non_ema variant from the diffusers/stable-diffusion-variants repository, you should specify="non-ema" to download the non_ema files.

### Loading variants

In [ ]:
from diffusers import DiffusionPipeline
import torch

repo_id="runwayml/stable-diffusion-v1-5"

# load fp16 variant
stable_diffusion = DiffusionPipeline.from_pretrained(repo_id, variant='fp16', torch_dtype=torch.float16)

# load non_ema variant
# stable_diffusion = DiffusionPipeline.from_pretrained(repo_id, variant='non_ema')

### Saving a variant

In [ ]:
# save as fp16 variant
stable_diffusion.save_pretrained(repo_id, variant='fp16')

# save as non-ema variant
# stable_diffusion.save_pretrained(repo_id, variant='non_ema')

#### Loading the fp16 variants

In [ ]:
stable_diffusion = DiffusionPipeline.from_pretrained(repo_id, variant='fp16', torch_dtype=torch.float16)

## Models

Models are loaded from the `ModelMixin.from_pretrained()` method, **which downloads and caches the latest version of the model weights and configurations.** If the latest files are available in the local cache, from_pretrained() reuses files in the cache instead of redownloading them.

Models can be loaded from a subfolder with the subfolder argument. For example, the model weights for runwayml/stable-diffusion-v1-5 are stored in the unet subfolder:

#### Loading the `unet` model weights

In [ ]:
from diffusers import UNet2DConditionModel

model=UNet2DConditionModel.from_pretrained(repo_id, subfolder='unet')

#### Loading from a repository's directory

In [ ]:
from diffusers import UNet2DModel

repo2_id="google/ddpm-cifar10-32"

model=UNet2DModel.from_pretrained(repo2_id,)

## Schedulers

Schedulers are loaded from the `SchedulerMixin.from_pretrained()` method, and unlike models, schedulers are not `parameterized` or `trained`; they are defined by a configuration file.

**Loading schedulers does not consume any significant amount of memory and the same configuration file can be used for a variety of different schedulers.**

In [ ]:
from diffusers import StableDiffusionPipeline
from diffusers import (
    DDPMScheduler,
    DDIMScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler,
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    DPMSolverMultistepScheduler,
)

ddpm = DDPMScheduler.from_pretrained(repo_id, subfolder='scheduler')
ddim = DDIMScheduler.from_pretrained(repo_id, subfolder='scheduler')
pndm = PNDMScheduler.from_pretrained(repo_id, subfolder='scheduler')
lms = LMSDiscreteScheduler.from_pretrained(repo_id, subfolder='scheduler')
euler = EulerDiscreteScheduler.from_pretrained(repo_id, subfolder='scheduler')
euler_ancestral = EulerAncestralDiscreteScheduler.from_pretrained(repo_id, subfolder='scheduler')
dpm_multistep = DPMSolverMultistepScheduler.from_pretrained(repo_id, subfolder='scheduler')

pipeline = StableDiffusionPipeline.from_pretrained(repo_id, scheduler="<any of the scheduler's name below'>", torch_dtype=torch.float16)

### DiiffusionPipeline explained

As a class method, DiffusionPipeline.from_pretrained() is responsible for two things:
* Download the latest version of the folder strcuture required for inference and cache it. If the latest folder structure is avaliable inthe local cache, DiffusionPipeline.from_pretrained() reuses files in the cache instead of redownloading them.
* Load the cached weights into the correct pipline class, retrieved from the model_index.json file and return an instance of it.


The pipelines underlying folder structure corresponds directly with their class instances. i.e. the StableDiffusionPipeline corresponds to the folder structure in runwayml/stable-diffusion-v1-5.

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
print(pipeline)

You will see pipeline is an insrance of StableDiffusionPipline, which consists of seven components:

* "feature_extractor": a CLIPFeatureExtractor from transformers
* "safety_checker": a component from screening against harmful content
* "scheduler": an instance of PNDMScheduler
* "text_encoder": a CLIPTextEncoder from transformers
* "tokenizer": a CLIPTokenizer from transformers
* "unet": an instance of UNet2DConditionModel
* "vae": an instance of AutoencoderKL

You can access each of the components of the pipeline as an attribute to view its configuration:

In [ ]:
pipeline.tokenizer

Every pipeline expects a model_index.json file that tells the DiffusionPipeline:
* which pipeline class to load from _class_ name
* which version of Diffusers was used to create the model in _diffusers_ version
* what compoents from which library are stored in the subfolders(name corresponds to the component and subfolder name, library corresponds to the name of the library to load the class from, and class corresponds to the class name)